# Video wrapper understanding

In [2]:
# Let's create a tiny synthetic video and then run the exact workflow you showed.
# This way you can see how cv2.VideoCapture + CAP_PROP_FRAME_COUNT behaves in practice.

import cv2
import numpy as np
import os, sys
from pathlib import Path

def create_dummy_video(out_path: str, w=320, h=240, fps=15, frames=45, codec='mp4v'):
    """Create a small synthetic video to test VideoCapture workflow."""
    fourcc = cv2.VideoWriter_fourcc(*codec)
    vw = cv2.VideoWriter(out_path, fourcc, fps, (w, h))
    if not vw.isOpened():
        raise RuntimeError(f"Failed to open VideoWriter for {out_path} with codec {codec}")
    for i in range(frames):
        # simple moving gradient pattern
        img = np.zeros((h, w, 3), dtype=np.uint8)
        img[:] = ((i*5) % 255, (i*3) % 255, (i*7) % 255)
        cv2.putText(img, f"frame {i}", (20, h//2), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
        vw.write(img)
    vw.release()
    return out_path

# Try mp4 first; fallback to avi if mp4 isn't supported in this environment.
base = "."  # Use current directory instead of Linux-specific path
mp4_path = os.path.join(base, "dummy_test_video.mp4")
avi_path = os.path.join(base, "dummy_test_video.avi")

video_path = None
try:
    video_path = create_dummy_video(mp4_path, codec='mp4v')
    print("✅ Created MP4 video")
except Exception as e:
    print(f"⚠️  MP4 failed ({e}), trying AVI...")
    try:
        video_path = create_dummy_video(avi_path, codec='XVID')
        print("✅ Created AVI video")
    except Exception as e2:
        print(f"❌ Both MP4 and AVI failed: {e2}")
        raise

print("Created video at:", video_path)

# Now run the snippet-like workflow
print("🎬 Opening video file:", video_path)
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("❌ Could not open the video file.")
else:
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print("✅ Video opened")
    print("   - Resolution:", f"{width}x{height}")
    print("   - FPS:", fps)
    print("   - Total frames reported:", frame_count)
    # Peek a couple of frames to show reading works
    grabbed_frames = 0
    sample_idxs = [0, max(0, frame_count//2), max(0, frame_count-1)]
    for idx in sample_idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        print(f"   - Seek to frame {idx}: {'OK' if ret else 'FAIL'}")
        if ret:
            grabbed_frames += 1
    cap.release()
    print("Done. Frames successfully grabbed:", grabbed_frames)


✅ Created MP4 video
Created video at: .\dummy_test_video.mp4
🎬 Opening video file: .\dummy_test_video.mp4
✅ Video opened
   - Resolution: 320x240
   - FPS: 15.0
   - Total frames reported: 45
   - Seek to frame 0: OK
   - Seek to frame 22: OK
   - Seek to frame 44: OK
Done. Frames successfully grabbed: 3


In [3]:
# Test reading the video we just created
print("🔍 Testing video reading...")
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("❌ Failed to open video for testing")
else:
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    print(f"✅ Video properties:")
    print(f"   - Frames: {frame_count}")
    print(f"   - FPS: {fps}")
    print(f"   - Resolution: {width}x{height}")
    
    # Test reading a few frames
    frames_read = 0
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Go to first frame
    for i in range(min(5, frame_count)):  # Read up to 5 frames
        ret, frame = cap.read()
        if ret:
            frames_read += 1
        else:
            break
    
    print(f"   - Successfully read {frames_read} frames")
    
# Always release the capture
cap.release()
print("🏁 Video capture released")


🔍 Testing video reading...
✅ Video properties:
   - Frames: 45
   - FPS: 15.0
   - Resolution: 320x240
   - Successfully read 5 frames
🏁 Video capture released


# Clever sampling

In [4]:
import torch
print(torch.cuda.is_available(), torch.version.cuda)


True 12.1
